<a href="https://colab.research.google.com/github/drihn/CCMACLRL_EXAM/blob/main/Midterm_Exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
dataset_url = 'https://github.com/robitussin/CCMACLRL_EXAM/blob/a46a4e2a001dedaefc9b431d480b508ce86c2d96/datasets/train.csv?raw=true'
df = pd.read_csv(dataset_url)
df.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            188533 non-null  int64 
 1   brand         188533 non-null  object
 2   model         188533 non-null  object
 3   model_year    188533 non-null  int64 
 4   milage        188533 non-null  int64 
 5   fuel_type     183450 non-null  object
 6   engine        188533 non-null  object
 7   transmission  188533 non-null  object
 8   ext_col       188533 non-null  object
 9   int_col       188533 non-null  object
 10  accident      186081 non-null  object
 11  clean_title   167114 non-null  object
 12  price         188533 non-null  int64 
dtypes: int64(4), object(9)
memory usage: 18.7+ MB


In [4]:
df.isnull().sum()

id                  0
brand               0
model               0
model_year          0
milage              0
fuel_type        5083
engine              0
transmission        0
ext_col             0
int_col             0
accident         2452
clean_title     21419
price               0
dtype: int64

In [5]:
df.dropna(inplace=True)

In [6]:
df.isnull().sum()

id              0
brand           0
model           0
model_year      0
milage          0
fuel_type       0
engine          0
transmission    0
ext_col         0
int_col         0
accident        0
clean_title     0
price           0
dtype: int64

In [7]:
df.describe()

,id,model_year,milage,price
count,162610.000000,162610.000000,162610.000000,1.626100e+05
mean,94197.039137,2015.158822,71105.487086,4.081699e+04
std,54467.770815,5.699531,50157.874111,7.653857e+04
min,0.000000,1974.000000,100.000000,2.000000e+03
25%,47026.250000,2012.000000,29000.000000,1.570000e+04
50%,94139.000000,2016.000000,64300.000000,2.850000e+04
75%,141421.750000,2019.000000,101000.000000,4.699900e+04
max,188532.000000,2024.000000,405000.000000,2.954083e+06


In [8]:
df.sample(20)

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
26610,26610,Porsche,Macan GTS,2021,9989,Gasoline,2.9L V6 24V GDI DOHC Twin Turbo,7-Speed Automatic with Auto-Shift,Chalk,Black,None reported,Yes,66400
24005,24005,Lexus,RX 350 Base,2012,134000,Gasoline,275.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,A/T,White,Beige,At least 1 accident or damage reported,Yes,8500
107618,107618,Ford,Transit-350 XLT,2017,53000,Gasoline,275.0HP 3.7L V6 Cylinder Engine Gasoline Fuel,A/T,White,Gray,At least 1 accident or damage reported,Yes,23800
28593,28593,BMW,428 i,2014,91000,Gasoline,240.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,A/T,Blue,Black,At least 1 accident or damage reported,Yes,12500
166050,166050,INFINITI,QX60 Base,2016,69000,Gasoline,265.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,A/T,Gray,Black,None reported,Yes,19000
19451,19451,Chevrolet,Tahoe RST,2023,162000,Gasoline,355.0HP 5.3L 8 Cylinder Engine Gasoline Fuel,6-Speed A/T,White,Black,None reported,Yes,53000
100813,100813,BMW,740 i,2017,98600,Gasoline,320.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,8-Speed A/T,Black,Black,None reported,Yes,31000
35444,35444,Lincoln,MKZ Base,2015,70700,Gasoline,240.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,6-Speed A/T,Black,Black,At least 1 accident or damage reported,Yes,26999
170218,170218,Chevrolet,Malibu LTZ,2008,169500,Gasoline,255.0HP 3.6L V6 Cylinder Engine Gasoline Fuel,A/T,White,Gray,None reported,Yes,35500
171904,171904,Mercedes-Benz,S-Class S 550 4MATIC,2017,8984,Gasoline,3.0L I6 24V GDI DOHC Turbo,9-Speed Automatic,White,Black,None reported,Yes,23600


In [9]:
from datetime import datetime

current_year = datetime.now().year
df['model_year'] = current_year - df['model_year'].astype(int)
df.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,MINI,Cooper S Base,17,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,1,Lincoln,LS V8,22,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,2,Chevrolet,Silverado 2500 LT,22,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,3,Genesis,G90 5.0 Ultimate,7,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,4,Mercedes-Benz,Metris Base,3,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


In [10]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
label_encoder = LabelEncoder()
df['brand'] = label_encoder.fit_transform(df['brand'])
df['model'] = label_encoder.fit_transform(df['model'])
df['fuel_type'] = label_encoder.fit_transform(df['fuel_type'])
df['engine'] = label_encoder.fit_transform(df['engine'])
df['transmission'] = label_encoder.fit_transform(df['transmission'])
df['ext_col'] = label_encoder.fit_transform(df['ext_col'])
df['int_col'] = label_encoder.fit_transform(df['int_col'])
df['accident'] = label_encoder.fit_transform(df['accident'])
df['clean_title'] = label_encoder.fit_transform(df['clean_title'])

In [11]:
df

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,31,493,17,213000,2,110,38,306,71,1,0,4200
1,1,28,925,22,143250,2,358,38,257,10,0,0,4999
2,2,9,1567,22,136731,1,627,38,37,71,1,0,13900
3,3,16,755,7,19500,2,846,49,28,14,1,0,45000
4,4,36,1072,3,7388,2,252,23,28,10,1,0,97500
...,...,...,...,...,...,...,...,...,...,...,...,...,...
188527,188527,9,402,25,110000,2,604,38,298,71,1,0,14500
188528,188528,8,601,7,49000,2,849,49,298,10,1,0,27500
188529,188529,36,204,6,28600,2,757,31,298,14,0,0,30000
188530,188530,36,221,3,13650,2,904,23,298,14,1,0,86900


In [12]:
y_scaled = df['price']
y_scaled

0          4200
1          4999
2         13900
3         45000
4         97500
          ...  
188527    14500
188528    27500
188529    30000
188530    86900
188532    28995
Name: price, Length: 162610, dtype: int64

In [13]:
x = df.drop(['price'],axis=1)
x

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title
0,0,31,493,17,213000,2,110,38,306,71,1,0
1,1,28,925,22,143250,2,358,38,257,10,0,0
2,2,9,1567,22,136731,1,627,38,37,71,1,0
3,3,16,755,7,19500,2,846,49,28,14,1,0
4,4,36,1072,3,7388,2,252,23,28,10,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
188527,188527,9,402,25,110000,2,604,38,298,71,1,0
188528,188528,8,601,7,49000,2,849,49,298,10,1,0
188529,188529,36,204,6,28600,2,757,31,298,14,0,0
188530,188530,36,221,3,13650,2,904,23,298,14,1,0


1. Use the Dataset file to train your model. Split this into train and test to calculate the RMSE

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y_scaled, test_size=0.2, random_state=42)

In [15]:
model = LinearRegression()

In [16]:
model.fit(X_train, y_train)


LinearRegression()

In [17]:
y_pred = model.predict(X_test)


In [18]:
mean_squared_error(y_test, y_pred, squared=False)

79047.5545875024

In [19]:
test_url = 'https://github.com/robitussin/CCMACLRL_EXAM/blob/a46a4e2a001dedaefc9b431d480b508ce86c2d96/datasets/test.csv?raw=true'
dt = pd.read_csv(test_url)

In [20]:
sample_submission_url = 'https://github.com/robitussin/CCMACLRL_EXAM/blob/a46a4e2a001dedaefc9b431d480b508ce86c2d96/datasets/sample_submission.csv?raw=true'
sf = pd.read_csv(sample_submission_url)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 162610 entries, 0 to 188532
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype
---  ------        --------------   -----
 0   id            162610 non-null  int64
 1   brand         162610 non-null  int64
 2   model         162610 non-null  int64
 3   model_year    162610 non-null  int64
 4   milage        162610 non-null  int64
 5   fuel_type     162610 non-null  int64
 6   engine        162610 non-null  int64
 7   transmission  162610 non-null  int64
 8   ext_col       162610 non-null  int64
 9   int_col       162610 non-null  int64
 10  accident      162610 non-null  int64
 11  clean_title   162610 non-null  int64
 12  price         162610 non-null  int64
dtypes: int64(13)
memory usage: 17.4 MB


In [22]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125690 entries, 0 to 125689
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            125690 non-null  int64 
 1   brand         125690 non-null  object
 2   model         125690 non-null  object
 3   model_year    125690 non-null  int64 
 4   milage        125690 non-null  int64 
 5   fuel_type     122307 non-null  object
 6   engine        125690 non-null  object
 7   transmission  125690 non-null  object
 8   ext_col       125690 non-null  object
 9   int_col       125690 non-null  object
 10  accident      124058 non-null  object
 11  clean_title   111451 non-null  object
dtypes: int64(3), object(9)
memory usage: 11.5+ MB


In [23]:
from datetime import datetime

current_year = datetime.now().year
dt['model_year'] = current_year - dt['model_year'].astype(int)
dt.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title
0,188533,Land,Rover LR2 Base,9,98000,Gasoline,240.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,6-Speed A/T,White,Beige,None reported,Yes
1,188534,Land,Rover Defender SE,4,9142,Hybrid,395.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,8-Speed A/T,Silver,Black,None reported,Yes
2,188535,Ford,Expedition Limited,2,28121,Gasoline,3.5L V6 24V PDI DOHC Twin Turbo,10-Speed Automatic,White,Ebony,None reported,NaN
3,188536,Audi,A6 2.0T Sport,8,61258,Gasoline,2.0 Liter TFSI,Automatic,Silician Yellow,Black,None reported,NaN
4,188537,Audi,A6 2.0T Premium Plus,6,59000,Gasoline,252.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,A/T,Gray,Black,None reported,Yes


In [24]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
label_encoder = LabelEncoder()
dt['brand'] = label_encoder.fit_transform(dt['brand'])
dt['model'] = label_encoder.fit_transform(dt['model'])
dt['fuel_type'] = label_encoder.fit_transform(dt['fuel_type'])
dt['engine'] = label_encoder.fit_transform(dt['engine'])
dt['transmission'] = label_encoder.fit_transform(dt['transmission'])
dt['ext_col'] = label_encoder.fit_transform(dt['ext_col'])
dt['int_col'] = label_encoder.fit_transform(dt['int_col'])
dt['accident'] = label_encoder.fit_transform(dt['accident'])
dt['clean_title'] = label_encoder.fit_transform(dt['clean_title'])

In [25]:
id = sf.pop('id')
y_pred = model.predict(dt)

# Create a submission DataFrame
submission_df = pd.DataFrame({
    'id': id,
    'price': y_pred
})

# Save the submission DataFrame to a CSV file
submission_df.to_csv('submission_file.csv', index=False)
print("Submission file created: submission_file.csv")

Submission file created: submission_file.csv
